In [4]:
from typing import Dict

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.tools import Tool

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import FakeEmbeddings
from langchain_community.llms import FakeListLLM


In [5]:
fake_llm = FakeListLLM(
    responses=[
        "LangChain is a framework for building applications using large language models.",
        "It is important because it helps manage prompts, memory, tools, and external data.",
        "RAG improves accuracy by combining LLMs with external knowledge sources.",
        "Employees get 20 paid leaves per year."
    ]
)

parser = StrOutputParser()


In [6]:
explain_prompt = ChatPromptTemplate.from_template(
    "Explain {topic} clearly with one real-world example."
)

explain_chain = explain_prompt | fake_llm | parser


In [7]:
explain_chain.invoke({"topic": "LangChain"})


'LangChain is a framework for building applications using large language models.'

In [8]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}")
    ]
)

base_chat_chain = chat_prompt | fake_llm | parser

store: Dict[str, ChatMessageHistory] = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chat_chain = RunnableWithMessageHistory(
    base_chat_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)


In [9]:
chat_chain.invoke(
    {"input": "What is LangChain?"},
    config={"configurable": {"session_id": "user1"}}
)
chat_chain.invoke(
    {"input": "Why is it important?"},
    config={"configurable": {"session_id": "user1"}}
)


'RAG improves accuracy by combining LLMs with external knowledge sources.'

In [10]:
documents = [
    Document(page_content="LangChain enables modular LLM-powered applications."),
    Document(page_content="RAG improves factual accuracy using external knowledge."),
    Document(page_content="FAISS enables fast vector similarity search."),
]


In [11]:
embeddings = FakeEmbeddings(size=1536)

vector_db = FAISS.from_documents(documents, embeddings)
retriever = vector_db.as_retriever()


In [12]:
rag_prompt = ChatPromptTemplate.from_template(
    """
Use the context to answer the question.

Context:
{context}

Question:
{question}
"""
)

def rag_chain(question: str):
    docs = retriever.invoke(question)
    context = "\n".join(d.page_content for d in docs)
    return (rag_prompt | fake_llm | parser).invoke(
        {"context": context, "question": question}
    )


In [13]:
rag_chain("Why is RAG useful?")


'Employees get 20 paid leaves per year.'

In [14]:
def company_policy_api(query: str) -> str:
    policies = {
        "leave": "Employees get 20 paid leaves per year.",
        "work": "Hybrid work model.",
    }
    return policies.get(query.lower(), "Policy not found.")

policy_tool = Tool(
    name="CompanyPolicyAPI",
    func=company_policy_api,
    description="Fetches company HR policies"
)


In [15]:
def tool_chain(question: str):
    return company_policy_api(question)


In [16]:
tool_chain("What is the leave policy?")


'Policy not found.'